In [1]:
!pip install py_ecc
!pip install web3
!pip install ecdsa

### 生成對稱式鑰匙key (用於加密雲端上的資料)

In [1]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad

def encrypt(data, key):
    cipher = AES.new(key, AES.MODE_CBC)
    ciphertext = cipher.encrypt(pad(data.encode('utf-8'), AES.block_size))
    iv = cipher.iv
    return ciphertext, iv

def decrypt(ciphertext, key, iv):
    cipher = AES.new(key, AES.MODE_CBC, iv)
    decrypted_data = unpad(cipher.decrypt(ciphertext), AES.block_size)
    return decrypted_data.decode('utf-8')

In [2]:
# 生成 key
key = get_random_bytes(32)
print(key)

b's~\n\x178"\x996\xbb\xb3\x06\x1c3/\xd4=\xedO\xdf\xf4\x01\xcdLd\xdb\x93\x93\xcb\x89@\x08L'


In [3]:
# 轉成int，方便加密
key_as_int = int.from_bytes(key, byteorder='big')
print(key_as_int)

# 解密可以轉回bytes
key_as_bytes = key_as_int.to_bytes(32, byteorder='big')
print(key_as_bytes)

52238669960691673665825435312914707100888310462309790160327758673430785820748
b's~\n\x178"\x996\xbb\xb3\x06\x1c3/\xd4=\xedO\xdf\xf4\x01\xcdLd\xdb\x93\x93\xcb\x89@\x08L'


In [4]:
# 將key切成兩塊
# 為了符合橢圓曲線階數(BN256_ORDER)

part1 = key_as_int // (10 ** 38)
part2 = key_as_int % (10 ** 38)

print(f"部分1: {part1}")
print(f"部分2: {part2}")

部分1: 522386699606916736658254353129147071008
部分2: 88310462309790160327758673430785820748


In [5]:
# 測試拼回key
m_reconstructed = part1 * (10 ** 38) + part2
print(f"拼好的key: {m_reconstructed}")

assert m_reconstructed == key_as_int, "拼接失败"
print("拼接成功")

拼好的key: 52238669960691673665825435312914707100888310462309790160327758673430785820748
拼接成功


In [6]:
# 示例用法
data_to_encrypt = "Hello, World!"

key = key_as_bytes

# 加密
encrypted_data, iv = encrypt(data_to_encrypt, key)

# 解密
decrypted_data = decrypt(encrypted_data, key, iv)

print("key:", key)
print("Original Data:", data_to_encrypt)
print("Encrypted Data:", encrypted_data)
print("Decrypted Data:", decrypted_data)

key: b's~\n\x178"\x996\xbb\xb3\x06\x1c3/\xd4=\xedO\xdf\xf4\x01\xcdLd\xdb\x93\x93\xcb\x89@\x08L'
Original Data: Hello, World!
Encrypted Data: b'S\x8aK\x12\\$\xbe\x10\xa5\xe1\xcc\xef\xb3=Gu'
Decrypted Data: Hello, World!


### 橢圓曲線加密
-  m：秘密值，這是Alice想要隱藏的值。(假設是key)-  
M := mG：這是秘密的「指紋」，它是透過將秘密 m 與橢圓曲線上的基點 G 相乘來生成的。這相當於加密的m
-  a：Alice的私鑰。
-  A：Alice的公開金鑰，即 A = aG。
-  B：Bob選擇的隨機值 b 乘以基點 G，即 B = bG。bG。


In [8]:
from py_ecc.bn128 import G1, add, multiply, pairing
import os

# BN256 曲線的階數
BN256_ORDER = 21888242871839275222246405745257275088548364400416034343698204186575808495617
    
b = int.from_bytes(os.urandom(32), byteorder="big") % BN256_ORDER
#m = int.from_bytes(os.urandom(32), byteorder="big") % BN256_ORDER
#m = part1

# 1. 生成 B = b * G
B = multiply(G1, b)

# 2. 生成 M = m * G
M1 = multiply(G1, part1)
M2 = multiply(G1, part2)

# 3. 計算 c = (m + b)
c1 = (part1 + b) 
c2 = (part2 + b)

In [9]:
# 驗證 C = c*G 是否等於 M + B
C1 = multiply(G1, c1)

M_plus_B = add(M1, B)
assert C1 == M_plus_B, "驗證失敗"
print("驗證通過！")

驗證通過！


In [13]:
print(key_as_int)
print(c1-b)
print(c2-b)

66104605630294247364083747132565623191223255450467998860141861021580600638715
661046056302942473640837471325656231912
23255450467998860141861021580600638715


In [52]:
# M, B, c
M1_to_tuple = (int(M1[0]), int(M1[1]))
print("M1:", M1_to_tuple)
print()

M2_to_tuple = (int(M2[0]), int(M2[1]))
print("M2:", M2_to_tuple)
print()

B_to_tuple = (int(B[0]), int(B[1]))
print("B:",B_to_tuple)
print()

#C_to_tuple = (int(C1[0]), int(C1[1]))
#print("C:",C_to_tuple)

print("c1:", c1)
print("c2:", c2)

M1: (18479165454232861116319427502477770473672567350588763535693451224415387532385, 6404929343959435307309967611570020832608603063766868664163498276419700687739)

M2: (867737788080039067470717243321464069161115999208033877480298464187572825888, 8054430195105493732222839913139293945739730365195718698070643473826465280299)

B: (5378922130471260529185872467495839666835425194740989935753900070491847019445, 5273430464091934285569218017182043859047827619487480750699855161722102225912)

c1: 5447109968213176062286855625893016543261004463439169242128025057710130810166
c2: 5447109968213176062286855625893016543206849677472654571817213524835386810708


In [177]:
# 偷改變值
M_to_tuple = (int(M[0])+1, int(M[1]))

### 上傳 M，B，C 到智能合約上進行驗證

In [53]:
from web3 import Web3
import json

# 初始化 web3 连接
w3 = Web3(Web3.HTTPProvider("http://localhost:8545"))  # 替换为你的以太坊节点地址

# 合约地址和 ABI
contract_address = '0x6A24bdBC27dcC2269EAa16B6bF99AdBE0DB7f3bE'

# 加載 ABI 和 Bytecode
with open("VerifyStepM3.abi") as f:
    contract_abi = json.load(f)


# 获取合约实例
contract = w3.eth.contract(
    address=contract_address, 
    abi=contract_abi
)

#result = contract.functions.verify1(M_to_tuple, B_to_tuple, C_to_tuple).call()
#result = contract.functions.verify2(M_to_tuple, B_to_tuple, c1).call()
result = contract.functions.verify3(B_to_tuple, M1_to_tuple, c1, M2_to_tuple, c2).call()

print(f"Verification result: {result}")


Verification result: True


In [55]:
# verify2
# 發送交易
txn_hash = contract.functions.verify2(M1_to_tuple, B_to_tuple, c1).transact({
    'from': w3.eth.accounts[0],
    'gas': 2000000  
})

# 等待交易完成
txn_receipt = w3.eth.wait_for_transaction_receipt(txn_hash)

if txn_receipt['status'] == 1:
    print("Transaction successful!")
else:
    print("Transaction failed.")

# 獲取gas消耗
gas_used = txn_receipt['gasUsed']
print(f"Gas used: {gas_used}")

Transaction successful!
Gas used: 33768


In [54]:
# verify3
# 發送交易
txn_hash = contract.functions.verify3(B_to_tuple, M1_to_tuple, c1, M2_to_tuple, c2).transact({
    'from': w3.eth.accounts[0],
    'gas': 2000000  
})

# 等待交易完成
txn_receipt = w3.eth.wait_for_transaction_receipt(txn_hash)

if txn_receipt['status'] == 1:
    print("Transaction successful!")
else:
    print("Transaction failed.")

# 獲取gas消耗
gas_used = txn_receipt['gasUsed']
print(f"Gas used: {gas_used}")

Transaction successful!
Gas used: 44326


### 測試區

In [22]:
from ecdsa import SigningKey, SECP256k1
from ecdsa.ellipticcurve import Point

# 生成 SECP256k1 橢圓曲線的基點
curve = SECP256k1.curve
G = SECP256k1.generator

# Alice 的秘密值 m
m = SigningKey.generate(curve=SECP256k1).privkey.secret_multiplier

# Bob 的隨機數 b
b = SigningKey.generate(curve=SECP256k1).privkey.secret_multiplier

# Alice 計算 M = mG (這是用來公開驗證的值)
M = m * G

# Bob 計算 B = bG
B = b * G

# Alice 和 Bob 計算混合值 c = m + b
c = (m + b) % SECP256k1.order

# 計算 C = (m + b)G
C = c * G

def point_to_tuple(point: Point):
    return (hex(point.x()), hex(point.y()))

# 輸出 M、B 和 C 點的座標，這些將傳遞到 Solidity 進行驗證
print("公開比對用值 (M):", point_to_tuple(M))
print("Bob 的公開點 (B):", point_to_tuple(B))
print("混合結果點 (C):", point_to_tuple(C))


公開比對用值 (M): ('0x96532721f59d5bd5fb09f78d015d9de71827b54086e8ce6bc165041231e8946e', '0x69acd3c4a3d1b1d3bd6e8a30492800bd580bd164ce270a26a6a49968b67bc019')
Bob 的公開點 (B): ('0x8ec671e9ca4d4d76b6a268e206abbf445d058628d0116dcda2330b3f5c195e06', '0x569990b744caed288d0dcb78bebb89932e9fd770b0d93e38ef7aa6d72ca9ceea')
混合結果點 (C): ('0xff40ef2ad8fb4d9be016ae3afce9a60c8a82d4c7a3dd08d2f596e74177ce87d0', '0x43c154332da0569a4f4fd1f17c6762ce1a11df1eeddef3509020b265b439c05c')


In [17]:
from ecdsa import SigningKey, SECP256k1
from ecdsa.ellipticcurve import Point

# 生成 SECP256k1 椭圆曲线的基点 G
curve = SECP256k1.curve
G = SECP256k1.generator

# 定义加密函数: c = m * G + b * G
def ecc_encrypt(m, b):
    # m 和 b 是整数，表示为椭圆曲线上的点
    m_point = m * G  # m 对应的椭圆曲线点
    b_point = b * G  # b 对应的椭圆曲线点
    
    # 加密: c = m_point + b_point
    c_point = m_point + b_point
    
    return c_point

# 定义解密函数: m = c - b * G (通过加上负的b * G实现减法)
def ecc_decrypt(c_point, b):
    # 计算 -b * G
    neg_b_point = -b * G
    # 恢复 m 的椭圆曲线点: m_point = c_point + (-b_point)
    m_point = c_point + neg_b_point
    
    return m_point
    
# 测试代码
if __name__ == "__main__":
    # 原始数据 m 和 b
    m = 123456789  # m 是一个整数，模拟为标量
    b = 987654321  # b 是一个整数，模拟为标量

    print(f"原始 m: {m}")
    print(f"原始 b: {b}")

    # 加密过程
    c_point = ecc_encrypt(m, b)
    print(f"加密后的椭圆曲线点 c: ({c_point.x()}, {c_point.y()})")

    # 解密过程
    decrypted_m_point = ecc_decrypt(c_point, b)
    print(f"解密后的椭圆曲线点 m: ({decrypted_m_point.x()}, {decrypted_m_point.y()})")

    # 验证解密结果是否正确
    # m_point 应该等于 m * G
    expected_m_point = m * G
    assert decrypted_m_point == expected_m_point, "解密失败"
    print("解密成功，m 的点与预期相符！")


原始 m: 123456789
原始 b: 987654321
加密后的椭圆曲线点 c: (103213092766970105549724903803005786552161881386844729536135103219576752988032, 106554927841606769585597959475808245486908341069295995421064383444758127827491)
解密后的椭圆曲线点 m: (4051293998585674784991639592782214972820158391371785981004352359465450369227, 88166831356626186178414913298033275054086243781277878360288998796587140930350)
解密成功，m 的点与预期相符！


In [20]:
expected_m_point

In [21]:
decrypted_m_point

In [3]:
def l(number):
    bit_length = number.bit_length()
    print(f"The number has {bit_length} bits.")
#l(int(M[0]))

In [5]:
BN256_ORDER = 21888242871839275222246405745257275088548364400416034343698204186575808495617
l(BN256_ORDER)

The number has 254 bits.
